<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/Code/ReHealthCo_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo Prediction-Prozess

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

## Importieren der notwendigen Libraries: sqlite3, csv, pandas

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
import numpy as np
from pandas.util import hash_pandas_object
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

### Erstellung einer virtuellen Kopie des GitHub Repositorium DMA2023TeamD in Colab

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD.git

In [ ]:
# Wechsel in den Unterordner csv_data des Github-Projekt-Ordners
%cd /content/DMA2023TeamD/csv_data

In [ ]:
# Auflistung aller Ordnerinhalte
!ls

### Benennung des rhc-DataFrames und Einlesen des entsprechenden anonymisierten csv-Files von Github

In [ ]:
# Pfad-Abkürzung zum Speicherort der anonymisierten csv-Tabelle
csv_path_anonymrhc = "/content/DMA2023TeamD/csv_data"

In [ ]:
# Einlesen der anonymisierten Tabelle als Pandas DataFrame
rhc_df = pd.read_csv(f"{csv_path_anonymrhc}/anonym_rhcpat.csv")

In [ ]:
# Anzeige des DF-Inhalts
rhc_df

# Einlesen passender DataFrames (DF) nach Auswertung durch den EDA-Prozess

### Ziel-DF rhc_all_df mit allen geplanten Vergleichen ohne Bereinigungen

In [ ]:
diag_anz_df = rhc_df[["RHC_PSEUDONYM", "DESC_CON"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("DESC_CON", ascending=False) # Erstellung DF diag_anz_df mit Spalte Diagnosen/Patient als Name
diag_anz_df.rename(columns={"DESC_CON": "DESC_CON_ANZ"}, inplace=True) # Umbenunng der Spalte Diagnosen/Patient in DESC_CON_ANZ
rhc_diagn_df = pd.DataFrame.merge(rhc_df, diag_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left") # Spalte Diagnosen/Patient an das gesamte DataFrame hängen
rhc_diagn_df.loc[:, 'DESC_CON_SORT'] = rhc_diagn_df.loc[:, 'DESC_CON_ANZ'] # Spalte DESC_CON_ANZ verdoppeln
rhc_diagn_df['DESC_CON_SORT'].values[rhc_diagn_df['DESC_CON_SORT'] > 2] = 3 # Ersetzen aller Werte bei Patienten > 2 Diagnosen mit einer 3
rhc_diagn_df["HEALTHCARE_EXPENSES_UNCOVERED"] = rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]-rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"] # neue Spalte HEALTHCARE_EXPENSES uncovered
rhc_diagn_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"] = rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]/rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"] # neue Spalte Verhältnis Ausgaben abgedeckt/nicht abgedeckt mit Spaltennahmen VERH_HEALTHCARE_EXPENSES_COV_UNCOV
rhc_enc_anz_df = rhc_diagn_df[["RHC_PSEUDONYM", "ENCOUNTERCLASS_ENC"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("ENCOUNTERCLASS_ENC", ascending=False) # Anzeige Encounterbegegnungen/Patient, absteigend sortiert
rhc_enc_anz_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_ENC_ANZ"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Patient in ENCOUNTERCLASS_ENC_ANZ
rhc_all_df = pd.DataFrame.merge(rhc_diagn_df, rhc_enc_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left") # Spalte Encounterbegegnungen/Patient an das gesamte DataFrame hängen
rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ_SORT'] = rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ'] # Spalte ENCOUNTERCLASS_ENC_ANZ verdoppeln
rhc_all_df['ENCOUNTERCLASS_ENC_ANZ_SORT'].values[rhc_all_df['ENCOUNTERCLASS_ENC_ANZ_SORT'] > 2] = 3 # Ersetzen aller Werte bei Patienten > 2 Encounterbegegnungen mit einer 3
rhc_all_df # Anzeige des Ergebnis

### Ziel-DF rhc_all_nowell_df mit allen geplanten Vergleichen ohne wellness bei Encounterbegegnungen sowie Umschreibung aller Stringwerte in Integer zur Durchführung von Berechnungen

In [ ]:
rhc_all_nowell_df = rhc_all_df.drop(rhc_all_df[ (rhc_all_df['ENCOUNTERCLASS_ENC'] == "wellness")].index) # Löschen der Encounterbegegnung wellness 
# Umschreibung aller string-werte in integer vor Gruppierung auf einzelnen Patienten
rhc_all_nowell_df.ENCOUNTERCLASS_ENC.replace(['ambulatory', 'inpatient', 'emergency', 'outpatient'], [0, 1, 2, 3], inplace=True)
rhc_all_nowell_df.DATENBANKART.replace(['asthma', 'covid', 'metabolic'], [0, 1, 2], inplace=True)
rhc_all_nowell_df.DESC_CON.replace(['COVID-19', 'Coronary Heart Disease', 'Pneumonia (disorder)', "Hypoxemia (disorder)", "Acute respiratory failure (disorder)", "Cardiac Arrest", "Atrial Fibrillation", "Stroke", "Sepsis caused by virus (disorder)", 
                                        "Acute deep venous thrombosis (disorder)", "Acute pulmonary embolism (disorder)", "Myocardial Infarction", "Septic shock (disorder)", "Acute respiratory distress syndrome (disorder)",
                                        "Injury of heart (disorder)", "Acquired coagulation disorder (disorder)", "Childhood asthma", "Injury of kidney (disorder)", "Heart failure (disorder)", "Bacterial infectious disease (disorder)", "Asthma"
                                        ], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], inplace=True)
rhc_all_nowell_df

### Ziel-DF anz_pat_dign_enc_df mit Anzeige Patienten/Diagnose sowie Patienten/Encountbegegnung, absteigend sortiert nach Patienten/Diagnose

In [ ]:
pat_anz_nowellrechnung_df = rhc_df.drop(rhc_df[ (rhc_df['ENCOUNTERCLASS_ENC'] == "wellness") ].index) # Löschen der Encounterbegegnung wellness 
diagn_encnowell_df = pat_anz_nowellrechnung_df[["RHC_PSEUDONYM", "DESC_CON", "ENCOUNTERCLASS_ENC"]].groupby(["DESC_CON"]).count().sort_values("RHC_PSEUDONYM", ascending=False) # Benennung des DF
diagn_encnowell_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
anz_pat_dign_enc_df = diagn_encnowell_df.reset_index()
anz_pat_dign_enc_df.rename(columns={"RHC_PSEUDONYM": "PAT_ANZ", "DESC_CON": "DIAGNOSEN", "ENCOUNTERCLASS_NOWELL": "ENC_BEGEG_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
anz_pat_dign_enc_df.DIAGNOSEN.replace(['COVID-19', 'Coronary Heart Disease', 'Pneumonia (disorder)', "Hypoxemia (disorder)", "Acute respiratory failure (disorder)", "Cardiac Arrest", "Atrial Fibrillation", "Stroke", "Sepsis caused by virus (disorder)", 
                                        "Acute deep venous thrombosis (disorder)", "Acute pulmonary embolism (disorder)", "Myocardial Infarction", "Septic shock (disorder)", "Acute respiratory distress syndrome (disorder)",
                                        "Injury of heart (disorder)", "Acquired coagulation disorder (disorder)", "Childhood asthma", "Injury of kidney (disorder)", "Heart failure (disorder)", "Bacterial infectious disease (disorder)", "Asthma"
                                        ], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], inplace=True) # Umbenunng der Diagnosen in 0 bis 20 um damit zu rechnen
anz_pat_dign_enc_df # Anzeige DF

### Ziel-DF ohne Extremwerte basierend auf dem DF rhc_all_nowell_df mit allen geplanten Vergleichen ohne wellness bei Encounterbegegnungen

### folgende essentielle DF wurden erstellt: 
rhc_all_df  =  alles

rhc_all_nowell_df  =  alles ohne wellness

anz_pat_dign_enc_df  =  Anzahl Patienten/Diagnose sowie Patienten/Encountbegegnung

# Machinelearning Modelle

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_all_df.columns.tolist())

In [ ]:
# Prediction für DF alles
dfpred = rhc_all_df[["AGE", "GENDER_PAT", "HEALTHCARE_EXPENSES_PAT", "DESC_CON_ANZ", "ENCOUNTERCLASS_ENC_ANZ"]].groupby(["AGE", "GENDER_PAT"]).mean().reset_index()
dfpred

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_all_nowell_df.columns.tolist())

In [ ]:
# Prediction für DF alles ohne wellness
dfprednowell = rhc_all_nowell_df[["AGE", "GENDER_PAT", "HEALTHCARE_EXPENSES_PAT", "DESC_CON_ANZ", "ENCOUNTERCLASS_ENC_ANZ"]].groupby(["AGE", "GENDER_PAT"]).mean().reset_index()
dfprednowell

In [ ]:
# Anzeige der Kopfspalten des DF
print(anz_pat_dign_enc_df.columns.tolist())

In [ ]:
# Prediction für DF Anzahl Patienten/Diagnose sowie Patienten/Encountbegegnung
dfpredanz = anz_pat_dign_enc_df[['DIAGNOSEN', 'PAT_ANZ', 'ENC_BEGEG_NOWELL']].groupby(['DIAGNOSEN']).mean().reset_index()
dfpredanz

In [ ]:
# Mischen der Daten
dfpred = dfpred.sample(frac=1)
dfprednowell = dfprednowell.sample(frac=1)
dfpredanz = dfpredanz.sample(frac=1)

In [ ]:
# Splitten der Daten in train und test Dataframes
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

In [ ]:
# Ausgabe der Shapes von train und test Dataframes
dfpred_train, dfpred_test = train_test_split(dfpred, test_size=test_ratio, random_state=0)
dfprednowell_train, dfprednowell_test = train_test_split(dfprednowell, test_size=test_ratio, random_state=0)
dfpredanz_train, dfpredanz_test = train_test_split(dfpredanz, test_size=test_ratio, random_state=0)

In [ ]:
dfpred_train.shape, dfpred_test.shape

In [ ]:
dfprednowell_train.shape, dfprednowell_test.shape

In [ ]:
dfpredanz_train.shape, dfpredanz_test.shape

In [ ]:
print(dfpred_train)
print(dfprednowell_train)
print(dfpredanz_train)

Bestimmen der Features und des Zielwertes

In [ ]:
X = dfpred_train.drop("HEALTHCARE_EXPENSES_PAT", axis=1)
y = dfpred_train["HEALTHCARE_EXPENSES_PAT"]

In [ ]:
X1 = dfprednowell_train.drop("HEALTHCARE_EXPENSES_PAT", axis=1)
y1 = dfprednowell_train["HEALTHCARE_EXPENSES_PAT"]

In [ ]:
X2 = dfpredanz_train.drop("PAT_ANZ", axis=1)
y2 = dfpredanz_train["PAT_ANZ"]

In [ ]:
X2

In [ ]:
X3 = dfpredanz_train.drop("ENC_BEGEG_NOWELL", axis=1)
y3 = dfpredanz_train["ENC_BEGEG_NOWELL"]

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y.head()

In [ ]:
y.shape

In [ ]:
pred_test_df = pd.concat([X, y], axis=1)
pred_test_df.shape

In [ ]:
pred_test_nowell_df = pd.concat([X1, y1], axis=1)
pred_test_nowell_df.shape

In [ ]:
pred_test_anzpat_df = pd.concat([X2, y2], axis=1)
pred_test_anzpat_df.shape

In [ ]:
pred_test_anzenc_df = pd.concat([X3, y3], axis=1)
pred_test_anzenc_df.shape

In [ ]:
# shuffle
pred_test_df.sample(frac=1)

In [ ]:
# shuffle
pred_test_nowell_df.sample(frac=1)

In [ ]:
# shuffle
pred_test_anzpat_df.sample(frac=1)

In [ ]:
# shuffle
pred_test_anzenc_df.sample(frac=1)

In [ ]:
# split data to testing and training dataset alles
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # ACHTUNG auf 42 setzen!
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# split data to testing and training dataset alles ohne wellness
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42) # ACHTUNG auf 42 setzen!
X1_train.shape, y1_train.shape, X1_test.shape, y1_test.shape

In [ ]:
# split data to testing and training dataset Anzahl Patienten/Diagnose
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42) # ACHTUNG auf 42 setzen!
X2_train.shape, y2_train.shape, X2_test.shape, y2_test.shape

In [ ]:
# split data to testing and training dataset Anzahl Patienten/Encountbegegnung
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42) # ACHTUNG auf 42 setzen!
X3_train.shape, y3_train.shape, X3_test.shape, y3_test.shape

In [ ]:
# create an object
linreg = LinearRegression()
ElasticNet = ElasticNet()
RF = RandomForestRegressor()

### Machine learning modelle trainieren - es lässt sich immmer nur ein Model trainieren, daher je nach Model Code aus-/kommentieren

Training und Test für DF alles - ggf. aus-/kommentieren

In [ ]:
# ElasticNet
ElasticNet.fit(X_train, y_train)
test_pred_elasticnet = ElasticNet.predict(X_test)
test_pred_elasticnet

In [ ]:
# Lineare Regression
linreg.fit(X_train, y_train)
test_pred_lin = linreg.predict(X_test)
test_pred_lin

In [ ]:
# RandomForest
RF.fit(X_train, y_train)
test_pred_RF = RF.predict(X_test)
test_pred_RF

Training und Test für DF alles ohne wellness - ggf. aus-/kommentieren

In [ ]:
# # ElasticNet
# ElasticNet.fit(X1_train, y1_train)
# test_pred_elasticnet1 = ElasticNet.predict(X1_test)
# test_pred_elasticnet1

In [ ]:
# # Lineare Regression
# linreg.fit(X1_train, y1_train)
# test_pred_lin = linreg.predict(X1_test)
# test_pred_lin

In [ ]:
# # RandomForest
# RF.fit(X1_train, y1_train)
# test_pred_RF = RF.predict(X1_test)
# test_pred_RF

Training und Test für DF Anzahl Patienten/Diagnose - ggf. aus-/kommentieren

In [ ]:
# # ElasticNet
# ElasticNet.fit(X2_train, y2_train)
# test_pred_elasticnet1 = ElasticNet.predict(X2_test)
# test_pred_elasticnet1

In [ ]:
# # Lineare Regression
# linreg.fit(X2_train, y2_train)
# test_pred_lin = linreg.predict(X2_test)
# test_pred_lin

In [ ]:
# # RandomForest
# RF.fit(X2_train, y2_train)
# test_pred_RF = RF.predict(X2_test)
# test_pred_RF

Training und Test für DF Anzahl Patienten/Encounterbegegnung - ggf. aus-/kommentieren

In [ ]:
# # ElasticNet
# ElasticNet.fit(X3_train, y3_train)
# test_pred_elasticnet1 = ElasticNet.predict(X3_test)
# test_pred_elasticnet1

In [ ]:
# # Lineare Regression
# linreg.fit(X3_train, y3_train)
# test_pred_lin = linreg.predict(X3_test)
# test_pred_lin

In [ ]:
# # RandomForest
# RF.fit(X3_train, y3_train)
# test_pred_RF = RF.predict(X3_test)
# test_pred_RF

### Vorhersage am Beispiel der linearen Regression für Healthcare-Gesamtkosten für einen männlichen Patienten im Alter von 40 J, der bislang 1 Diagnose gestellt bekommen sowie noch keinen Krankenhausbesuch absolviert hat: Betrag in $
(1 Diagnose aus folgenden Kategorien: 'COVID-19', 'Coronary Heart Disease', 'Pneumonia (disorder)', "Hypoxemia (disorder)", "Acute respiratory failure (disorder)", "Cardiac Arrest", "Atrial Fibrillation", "Stroke", "Sepsis caused by virus (disorder)", "Acute deep venous thrombosis (disorder)", "Acute pulmonary embolism (disorder)", "Myocardial Infarction", "Septic shock (disorder)", "Acute respiratory distress syndrome (disorder)", "Injury of heart (disorder)", "Acquired coagulation disorder (disorder)", "Childhood asthma", "Injury of kidney (disorder)", "Heart failure (disorder)", "Bacterial infectious disease (disorder)", "Asthma")

In [ ]:
# Für DF alles - ggf. aus-/kommentieren
healthcarecost_pred = linreg.predict(pd.DataFrame({"AGE":[40], "GENDER_PAT":[0], "DESC_CON_ANZ":[1], "ENCOUNTERCLASS_ENC_ANZ":[0]}))
healthcarecost_pred

In [ ]:
# # Für DF alles ohne wellness - ggf. aus-/kommentieren
# healthcarecost_pred = linreg.predict(pd.DataFrame({"AGE":[40], "GENDER_PAT":[0], "DESC_CON_ANZ":[1], "ENCOUNTERCLASS_ENC_ANZ":[0]}))
# healthcarecost_pred

Vorhersage am Beispiel der linearen Regression für Anzahl an Patienten, die an Diagnose 15 (= Acquired coagulation disorder (disorder)) leiden und 1x eine Krankenhausbegegnung hatten

In [ ]:
# # Für DF Anzahl Patienten/Diagnose
# diagn_pred = linreg.predict(pd.DataFrame({'DIAGNOSEN':[15], 'ENC_BEGEG_NOWELL':[1]}))
# diagn_pred

Vorhersage am Beispiel der linearen Regression für Anzahl an Krankenhausbegenungen, die an Diagnose 15 (= Acquired coagulation disorder (disorder)) leiden in Abhängigkeit der Patientenanzahl

In [ ]:
# # Für DF Anzahl Patienten/Encounterbegegnung
# diagn_pred = linreg.predict(pd.DataFrame({'DIAGNOSEN':[15], 'PAT_ANZ':[15]}))
# diagn_pred

### Auswertung Vorhersagekraft je nach Testverfahren
je nach Vorhersage-DF Zeile 1/2/3/4 aus-/kommentieren

In [ ]:
linreg.score(X_test, y_test) # DF alles
# linreg.score(X1_test, y1_test) # DF alles ohne wellness
# linreg.score(X2_test, y2_test) # DF Anzahl Patienten/Diagnose
# linreg.score(X3_test, y3_test) # DF Anzahl Patienten/Encountbegegnung

In [ ]:
ElasticNet.score(X_test, y_test) # DF alles
# ElasticNet.score(X1_test, y1_test) # DF alles ohne wellness
# ElasticNet.score(X2_test, y2_test) # DF Anzahl Patienten/Diagnose
# ElasticNet.score(X3_test, y3_test) # DF Anzahl Patienten/Encountbegegnung

In [ ]:
RF.score(X_test, y_test) # DF alles
# RF.score(X1_test, y1_test) # DF alles ohne wellness
# RF.score(X2_test, y2_test) # DF Anzahl Patienten/Diagnose
# RF.score(X3_test, y3_test) # DF Anzahl Patienten/Encountbegegnung

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
csv.__version__

In [ ]:
sns.__version__

In [ ]:
# Überprüfen aller Variablen
# %pip freeze

# Calculate the Checksum of the Data Set

In [ ]:
rhc_df_hashes = hash_pandas_object(rhc_df)
diag_anz_df_hashes = hash_pandas_object(diag_anz_df)
rhc_diagn_df_hashes = hash_pandas_object(rhc_diagn_df)
rhc_enc_anz_df_hashes = hash_pandas_object(rhc_enc_anz_df)
rhc_all_df_hashes = hash_pandas_object(rhc_all_df)
rhc_all_nowell_df_hashes = hash_pandas_object(rhc_all_nowell_df)
pat_anz_nowellrechnung_df_hashes = hash_pandas_object(pat_anz_nowellrechnung_df)
diagn_encnowell_df_hashes = hash_pandas_object(diagn_encnowell_df)
anz_pat_dign_enc_df_hashes = hash_pandas_object(anz_pat_dign_enc_df)

In [ ]:
Liste_rhcpredall_hashes = [rhc_df_hashes, diag_anz_df_hashes, rhc_diagn_df_hashes, rhc_enc_anz_df_hashes, rhc_all_df_hashes, rhc_all_nowell_df_hashes, pat_anz_nowellrechnung_df_hashes, diagn_encnowell_df_hashes, anz_pat_dign_enc_df_hashes]

In [ ]:
print(Liste_rhcpredall_hashes)

## Value should be 
8997601563088961698

-3795146034381248713

2650178518257614002

-5022892886560696056

8245114429953816038

-3815936067771996962

-840543143014388252

1693621233367357631

-3811992772619237490

In [ ]:
print(rhc_df_hashes.sum())
print(diag_anz_df_hashes.sum())
print(rhc_diagn_df_hashes.sum())
print(rhc_enc_anz_df_hashes.sum())
print(rhc_all_df_hashes.sum())
print(rhc_all_nowell_df_hashes.sum())
print(pat_anz_nowellrechnung_df_hashes.sum())
print(diagn_encnowell_df_hashes.sum())
print(anz_pat_dign_enc_df_hashes.sum())